In [1]:
import pandas as pd
import numpy as np
import os 
from utils.dataframe_tools import filter_out_nan 

In [2]:
df = pd.read_csv('./Data/Test/merge_tls_test_01.csv')

In [3]:
list_col = df.columns.tolist()

In [7]:
test = 'frame_num' 
test1 = 'eth.dst.ig' 
test2 = 'eth.dst'
a = test.split('.') 
b = test1.split('.') 
c = test2.split('.')
print(a) 
print(b)
print(c)
print(c in b)

['frame_num']
['eth', 'dst', 'ig']
['eth', 'dst']
False


In [ ]:
def protocol_tree_extract(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    dict_fields = {} 
    for layer in list_layers: 
        for field in list_fields: 

    for field in list_fields: 
        s_field = field.split('.') 
        if len(s_field) == 1 and s_field[0] not in list_layers: # statistical information, like frame_num, reassembled_info 
            dict_fields['statistics'].append(s_field) 
            
        if s_field[0] in list_layers: # layer prefix 
            if s_field[0] not in dict_fields.keys(): 
                last_len = len(s_field) 
                dict_fields[s_field[0]].append(s_field) 
            

    return 

In [3]:
all_nan_cols = df.columns[df.isna().all()] 

In [6]:
df_nonan = df.drop(columns=all_nan_cols)

In [2]:
path_test_directory = './Data/Test/filter_nan' 
# path_tls_csv = os.path.join(path_test_directory, 'tls_test_01.pcapng.csv') 
# path_tls_reassemble_csv = os.path.join(path_test_directory, 'reassemble_tls_test_01.pcapng.csv') 
path_test_csv = os.path.join(path_test_directory, 'filter_nan_merge_tls_test_01.csv') 

In [3]:
df = pd.read_csv(path_test_csv, index_col='frame_num') 
df.shape 

(1959, 190)

In [4]:
mask_df = df.notnull() 

In [21]:
dict_true = {} 
for col_num in range(mask_df.shape[1]): 
    list_true_indices = list(mask_df[mask_df.iloc[:, col_num]].index) 
    dict_true[col_num] = list_true_indices  

# for col in mask_df.columns: 
#     list_true_indices = list(mask_df[mask_df.loc[:, col]].index) 
#     dict_true[col] = list_true_indices 

In [22]:
# continuous
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
last_key = 0
last_value = dict_true[last_key] 
list_col = []
for key, value in dict_true.items(): 
    if key == last_key: # init 
        dict_block['block'].append(block_flag) 
        list_col.append(key) 
        dict_block['columns'].append(list_col.copy()) 
        dict_block['rows'].append(dict_true[key]) 
    else: 
        if value == last_value: 
            dict_block['columns'][block_flag].append(key) 
        if value != last_value: 
            block_flag += 1 
            dict_block['block'].append(block_flag) 
            list_col.clear() 
            list_col.append(key) 
            dict_block['columns'].append(list_col.copy()) 
            dict_block['rows'].append(dict_true[key]) 
            last_key = key 
            last_value =value 


In [7]:
df_block = pd.DataFrame(dict_block)

In [23]:
from tqdm import tqdm 

block_values = dict_block['block'] 
columns_values = dict_block['columns'] 
rows_values = dict_block['rows'] 

list_sub_df = []

for block_name, columns, rows in tqdm(zip(block_values, columns_values, rows_values)): 
    subset_rows = df.loc[rows]
    sub_df = subset_rows.iloc[:, columns]
    list_sub_df.append(sub_df) 

41it [00:00, 826.97it/s]


In [7]:
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
list_col = [] 
list_record_col = []
for key, value in dict_true.items(): 
    if key not in list_record_col: 
        for ik, iv in dict_true.items(): 
            if iv == value: 
                if block_flag not in dict_block['block']: 
                    dict_block['block'].append(block_flag) 
                    dict_block['rows'].append(dict_true[key]) 
                list_col.append(ik) 
            if iv != value: 
                continue 
        dict_block['columns'].append(list_col.copy()) 
        list_record_col.extend(list_col.copy()) 
        list_col.clear()
        block_flag += 1 

In [ ]:
print(dict_block)

In [8]:
df_block = pd.DataFrame(dict_block)

In [9]:
true_indices = list(mask_df[mask_df['tls_handshake_server_point_len']].index) 

In [6]:
intesection_series = mask_df.all()